# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [24]:
# imports
from holoviews.streams import Params
import holoviews as hv
import panel as pn
pn.extension()
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
import numpy as np

In [16]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension()

In [17]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [18]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data - data calculated in rental_analysis.ipynb

In [19]:
# Getting the data from the top 10 expensive neighbourhoods
file_path1 = Path('Data/top10_expensive_data_fin.csv')
top10_expensive_data_fin = pd.read_csv(file_path1)

# Calculate the mean number of dwelling types units per year
file_path2 = Path('Data/dwelling_types_per_year.csv')
dwelling_type_year_df = pd.read_csv(file_path2)

# Calculate the average monthly shelter costs for owned and rented dwellings
file_path3 = Path("Data/monthly_shelter_cost.csv")
monthly_shelter_cost = pd.read_csv(file_path3)

# Calculate the top 10 suburbs by year by nighbourhood - for sunburst
file_path4 = Path("Data/top10_expensive_data_year_neighbourhood_fin.csv")
top10_expensive_yearly_df = pd.read_csv(file_path4)


# Calculate location data for map - df_location_neighbourhood_map
file_path5 = Path("Data/df_location_neighbourhood_map.csv")
df_location_neighbourhood = pd.read_csv(file_path5)

# Calculate oveereview of value per year per neighbourhood
file_path6 = Path('Data/neighbourhood_yr_data.csv')
neighbourhood_yr_data = pd.read_csv(file_path6) #neighbourhood_yr_data

# Calculate the mean number of dwelling types units per year -for hvplot
file_path7 = Path('Data/dwellingtype_year_data.csv')
dwellingtype_year_data = pd.read_csv(file_path7)

### Panel Visualization Functions

In [20]:
# hvplot.help('line') # review of options

In [32]:
def create_bar_chart(data, xvalue, yvalue, titletext, xlabel, ylabel):
    
    return data.hvplot.bar(x=xvalue, y=yvalue,  xlabel=xlabel, ylabel=ylabel, title=titletext, rot=45, colorbar=True)
    

In [22]:
def create_line_chart(data, xvalue, yvalue, titletext, xlabel, ylabel):

    return data.hvplot.line(x=xvalue, y=yvalue, xlabel=xlabel, ylabel=ylabel,title=titletext, rot=45, colorbar=True)
    

In [9]:
# Define Panel visualization functions
def neighbourhood_map(): 
    """Neighbourhood Map - this map shows the neighbourhood map 
    this map shows a scatter map using mapbox and plotly express. It only takes exact 
    values from the stored dataframe - not dynamic"""
   

    map_plot = px.scatter_mapbox(
    df_location_neighbourhood,
    lat="lat",
    lon="lon",
    title = 'Realestate Analysis of Toronto 2001 to 2016 ',
    size="average_house_value",
    color="average_house_value",
    zoom=9,
    height=800,
    width = 1000,
    hover_name=df_location_neighbourhood.index, 
    hover_data=df_location_neighbourhood,
    #hover_data=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"],
    color_continuous_scale=px.colors.cyclical.IceFire
    )


    
    return map_plot
    




def average_house_value():
    """Average house values per year."""
    avg_house_price = to_data['average_house_value'].groupby('year').mean()
    title = 'Average House Value in Toronto Per Year'
    width=600
    height=600
    colour='aliceblue'
    ylabel = 'Avg House Value'
    xlabel = 'Year'
    # call function to get the chart
    
    fig = px.line(avg_house_price, title=title, width=width, height=height)
    fig.update_layout(
        font_family="Courier New",
        font_color="black",
        title_font_color="Blue",

        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        ),
        title={
            'text': title,
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(
            tickangle = 90,
            title_text = ylabel,
            title_font = {"size": 16},
            title_standoff = 25)
    fig.update_xaxes(
            tickangle = 90,
            title_text = xlabel,
            title_font = {"size": 16},
            title_standoff = 25)
    fig.update_traces(marker_color=colour)    
    

    # call line graph function
    return fig




def monthly_shelter_values(rent_or_own):
    # Create two line charts, one to plot the monthly shelter costs for owned dwelling and other for rented dwellings per year
    
    colourlist=['#2990A0','#A0296A'] # set colour options list
    panelcolour='#2990A0'
    width=600 # figure width
    height=600 # figure height
    if rent_or_own == 'rent':
        
        
        # for monthly renting 
        column  = 'shelter_costs_rented'
        data = monthly_shelter_cost # get only rented column
        rent_or_own = 'Rented' # text for title
        panelcolour = colourlist[0] # select colour
        titletext = 'Average Monthly Shelter Cost for ' + rent_or_own.upper() + ' Dwellings in Toronto'
        xvalue = 'year'
        yvalue = column
        xlabel = 'Year'
        ylabel = 'Avg Monthly Shelter Costs'
        colour = 'r'
        width = 600
        height = 600
        # call line graph function 
        rent_or_own_graph=create_line_chart(data, xvalue, yvalue, titletext, xlabel, ylabel, width, height)



    if rent_or_own == 'own':
        # for monthly owning
        column  = 'shelter_costs_owned'
        data = monthly_shelter_cost # get only owned column
        data = data.set_index(neighbourhood, inplace=True)
        rent_or_own = 'Owned' # text for title
        panelcolour = colourlist[1] # select colour
        titletext = 'Average Monthly Shelter Cost for ' + rent_or_own.upper() + ' Dwellings in Toronto'
    
        xvalue = 'year'
        yvalue = column
        xlabel = 'Year'
        ylabel = 'Avg Monthly Shelter Costs'
        colour = 'b'
        width = 600
        height = 600
        # call line graph function 
        rent_or_own_graph=create_line_chart(data, xvalue, yvalue, titletext, xlabel, ylabel, colour, width, height)

    
    return rent_or_own_graph





    

def average_value_by_neighbourhood():
    """Average house values by neighbourhood by year."""
    
    neighbourhood_line_hvplot = neighbourhood_yr_data.hvplot(kind='line', x='year', y='average_house_value', groupby='neighbourhood', dynspread=True, datashade=True,  xlabel="Year", ylabel="Avg House Price", rot=90).opts(
    yformatter="$%.0f", title="Average House Value by Neighbourhood by Year", framewise=True)
    
    return neighbourhood_line_hvplot

def dwelling_type_per_neighbourhood():
    dwell_by_neighbourhood_hvplot = dwellingtype_year_data.hvplot(kind='bar', rot=90, groupby=['neighbourhood'], xlabel="Year", height=500, ylabel="Dwelling Type Units").opts(
    yformatter="%.0f", title="Number of Toronto Dwelling Types by Neighbourhood by Year", framewise=True)
    
    return dwell_by_neighbourhood_hvplot

def number_dwelling_types(year):
    """Number of dwelling types per year. Takes year as an input"""
    
    if year == 2001:
        panelcolour = '#2990A0'
        
    elif year == 2006:
        panelcolour = '#A0296A'
        
    elif year == 2011:
        panelcolour = '#8F4395'
        
    elif year == 2016:
        panelcolour = '#29A03C'
    row = dwelling_type_year_df[dwelling_type_year_df.year==year]    


    # set values and get chart    
  
    titletext = 'Dwelling Types in Toronto in' +  str(year)  
    height = 10
    width = 15
    xvalue = 'year'
    xlabel = 'Year'
    ylabel = 'Number of Dwellings'
    yvalue = ['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']
    data = row
    colour = 'year'
    width = 600
    hight= 600
    
   

     
    bar_dwelling_year = create_bar_chart(data, xvalue, yvalue, titletext, xlabel, ylabel, width, height)
  
    #bar_dwelling_year = create_bar_chart (data, xvalue, yvalue, titletext, xlabel, ylabel, colour, width, height)
  
    return bar_dwelling_year

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    try:
        fig = px.bar(neighbourhood_yr_data, x="neighbourhood", y="average_house_value",  title="Average House Values Toronto Neighbourhood per Year", color="average_house_value", facet_row="year", width=900, height=1400)
        fig.update_layout(
            font_family="Courier New",
            font_color="black",
            title_font_color="Black",

            font=dict(
                family="Courier New, monospace",
                size=18,
                color="RebeccaPurple"
            ),
            title={
                'text': "Average House Prices Toronto Per Neighbourhood Per Year",
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
        fig.update_yaxes(
                tickangle = 90,
                title_text = "Avg House Price",
                title_font = {"size": 16},
                title_standoff = 25)
        fig.update_xaxes(title_font_family="Arial")
    
    except:
        print('Exception in averag house value - check input values and nulls')
    
    return fig

def top_most_expensive_neighbourhoods():
    """Top 10 Most Expensive Neighbourhoods in Toronto"""
    
    titletext = 'Top 10 Expensive Neighbourhoods in Toronto'
    data = top10_expensive_data_fin    
    xvalue = 'neighbourhood'
    yvalue = 'average_house_value'
    xlabel = 'Neighbourhood'
    ylabel = 'Avg House Value - $Million'
    colour = '#1f77b4' 
    width = 600
    height = 600
    # call line graph function
    
    
    top10_expensive_plot=create_line_chart(data, xvalue, yvalue, titletext, xlabel, ylabel, width, height)

   
    
    
    return top10_expensive_plot

def sunburts_cost_analysis():
    """Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year."""
    
    df = top10_expensive_yearly_df
    fig = px.sunburst(df, path=['year', 'neighbourhood'], values='average_house_value', \
                  
        color='shelter_costs_owned',\
        color_continuous_scale='Blues')
    return fig
 

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [10]:

# declare lists years for interactive selection
list_of_years = [2001,2006, 2011, 2016]
# declare lists rent or own for monthly shelter view
list_rent_own = ['rent','own']

# Create the main dashboard
# create columns
welcome_column = pn.Column(
    "## Toronto Housing Analysis Overview", 
    neighbourhood_map(),
)

dwelling_types_column = pn.Column(
    "## Toronto Dwellings",
    interact(number_dwelling_types, year=list_of_years, name='Select a Year'),
    
)

neighbour1_column= pn.Column(
    average_house_value_snapshot(),
)

neighbour2_column= pn.Column(
    average_value_by_neighbourhood(),
)

neighbour3_column= pn.Column(
    dwelling_type_per_neighbourhood(),
)

neighbourhood_column = pn.Column(
    "## Toronto Housing per Neighbourhood",
    ("Neighbourhood House Snapshot", neighbour1_column), 
    ("Average Neighbourhood House Values", neighbour2_column),  
    ("Average Neighbourhood Dwelling Types", neighbour3_column),)
    


average_house_column = pn.Column(
    "## Toronto Dwelling Types per Year",
    average_house_value(),
    

)

shelter_column = pn.Column(
    "## Monthly Housing Shelter Costs for Renting or Owning",
    interact(monthly_shelter_values, rent_or_own=list_rent_own),
)

expensive_type_column = pn.Column(
    "## Most Expensive Housing in Toronto Neighbourhood",
    top_most_expensive_neighbourhoods(),
    sunburts_cost_analysis()
)




In [11]:
# Create a Title for the Dashboard


# Define a welcome text
dash_title = 'Toronto Realestate Analysis 2001-2016'
welcome_text = 'Welcome, this analysis will review Toronto dwellings\n by type, value, rental costs, across neighbourhoods, \nbetween 2001 and 2016. What you see here is a map of average house values by neighbourhood. Hover over each data point and you will see more information about the neighbourhood housing analysis.'

# Create a tab layout for the dashboard
dash_tabs = pn.Tabs(
    ("Welcome", welcome_column), 
    ("Dwelling Types", dwelling_types_column),  
    ("Neighbourhood", neighbourhood_column),
    ("Average House Value", average_house_column),
    ("Most Expensive Neighbourhoods", expensive_type_column),
    ('Monthly Shelter Costs', shelter_column),
)







## Serve the Panel Dashboard

In [12]:
dash_tabs.servable()


pn.extension('plotly')



Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Column(name='Select a Year')
            [0] Column
                [0] HTML(str)
                [1] DiscreteSlider(formatter='%d', name='year', options=[2001, 2006, 2011, 2016], value=2001)
            [1] Row
                [0] HoloViews(Bars, name='Select a Year')
    [2] Column
        [0] Markdown(str)
        [1] Str(tuple)
        [2] Str(tuple)
        [3] Str(tuple)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] Plotly(Figure)
    [5] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='rent_or_own', options=['rent', 'own'], value='rent')
            [1] Row
                [0] HoloViews(Curve, name='interactive01668')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [25]:
def plot_housing_tx(number_of_sales):
    housing_transactions = pd.DataFrame(
        {
            "years": np.random.randint(2010, 2019, number_of_sales),
            "sales": np.random.randint(53, 500, number_of_sales),
            "foreclosures": np.random.randint(10, 147, number_of_sales),
        }
    ).sort_values(["years", "sales"])

    return housing_transactions.hvplot.scatter(
        x="sales",
        y="foreclosures",
        c="years",
        colormap="viridis",
        title="Alleghany, PA Housing Transactions",
    )


# Render plot with Panel interactive widget
interact(plot_housing_tx, number_of_sales=100)

Column
    [0] Column
        [0] IntSlider(end=300, name='number_of_sales', start=-100, value=100, value_throttled=100)
    [1] Row
        [0] HoloViews(Scatter, name='interactive02247')

In [33]:
def number_dwelling_types(year):
    """Number of dwelling types per year. Takes year as an input"""
    
    if year == 2001:
        panelcolour = '#2990A0'
        
    elif year == 2006:
        panelcolour = '#A0296A'
        
    elif year == 2011:
        panelcolour = '#8F4395'
        
    elif year == 2016:
        panelcolour = '#29A03C'
    row = dwelling_type_year_df[dwelling_type_year_df.year==year]    


    # set values and get chart    
  
    titletext = 'Dwelling Types in Toronto in' +  str(year)  
    xvalue = 'year'
    xlabel = 'Year'
    ylabel = 'Number of Dwellings'
    yvalue = ['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house']
    data = row
    colour = 'year'
    
   

     
    return create_bar_chart(data, xvalue, yvalue, titletext, xlabel, ylabel)
  


list_of_years = [2001,2006, 2011, 2016]
interact(number_dwelling_types, year=list_of_years)



Column
    [0] Column
        [0] DiscreteSlider(formatter='%d', name='year', options=[2001, 2006, 2011, 2016], value=2001)
    [1] Row
        [0] HoloViews(Bars, name='interactive03470')

## References

https://medium.com/@philipp.jfr/panel-announcement-2107c2b15f52
    
https://panel.holoviz.org/gallery/apis/stocks_hvplot.html 
    
https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289

https://www.youtube.com/watch?v=am0iMNa9-1M